In [50]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [51]:
import sklearn.datasets
wine = sklearn.datasets.load_wine()
wine.data.shape

(178, 13)

In [52]:
import pandas as pd
pd.DataFrame(data = np.c_[wine['data'], wine['target']],
                     columns= wine['feature_names'] + ['target'])

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2.0


In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine.data, 
    wine.target, 
    test_size=0.3, 
    shuffle=True)

X_train_ten = torch.FloatTensor(X_train)
X_test_ten = torch.FloatTensor(X_test)
y_train_ten = torch.LongTensor(y_train)
y_test_ten = torch.LongTensor(y_test)

In [54]:
class WineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(WineNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(13, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
wine_net = WineNet(20)

In [55]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
wine_net = wine_net.to(device)

X_train_ten = X_train_ten.to(device)
y_train_ten = y_train_ten.to(device)

X_test_ten = X_test_ten.to(device)
y_test_ten = y_test_ten.to(device)

In [56]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(wine_net.parameters(), 
                             lr=1.0e-3)

In [59]:
batch_size = 10

for epoch in range(5000):
    order = np.random.permutation(len(X_train_ten))
    for start_index in range(0, len(X_train_ten), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train_ten[batch_indexes]
        y_batch = y_train_ten[batch_indexes]
        
        preds = wine_net.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = wine_net.forward(X_test_ten)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test_ten).float().mean())

tensor(0.4074, device='cuda:0')
tensor(0.7222, device='cuda:0')
tensor(0.7037, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9259, device='cuda:0')
tensor(0.9259, device='cuda:0')
tensor(0.9074, device='cuda:0')
tensor(0.9630, device='cuda:0')
tensor(0.9630, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9259, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9074, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0.9444, device='cuda:0')
tensor(0

In [63]:
wine_net.inference(X_test_ten)

tensor([[9.9996e-01, 4.4197e-05, 4.4823e-14],
        [1.1557e-10, 5.8854e-04, 9.9941e-01],
        [3.3512e-02, 9.6649e-01, 4.4752e-07],
        [9.9995e-01, 4.5578e-05, 4.6922e-14],
        [3.3154e-02, 9.6685e-01, 4.5059e-07],
        [3.4346e-02, 9.6565e-01, 4.4058e-07],
        [9.9996e-01, 4.3887e-05, 4.4341e-14],
        [1.2684e-10, 6.3466e-04, 9.9937e-01],
        [2.8250e-02, 9.7175e-01, 4.9870e-07],
        [1.1527e-02, 9.8847e-01, 8.8161e-07],
        [3.9730e-10, 1.5972e-03, 9.9840e-01],
        [1.2119e-10, 6.1166e-04, 9.9939e-01],
        [9.9996e-01, 4.3604e-05, 4.3899e-14],
        [6.5006e-02, 9.3499e-01, 3.0874e-07],
        [1.2140e-10, 6.1250e-04, 9.9939e-01],
        [2.9452e-02, 9.7055e-01, 4.8571e-07],
        [9.9996e-01, 4.4742e-05, 4.5674e-14],
        [9.9996e-01, 4.3594e-05, 4.3883e-14],
        [4.3673e-10, 1.7266e-03, 9.9827e-01],
        [9.9996e-01, 4.3599e-05, 4.3892e-14],
        [5.1727e-02, 9.4827e-01, 3.5186e-07],
        [9.9996e-01, 4.4521e-05, 4